In [4]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("004. Crucero").getOrCreate()

In [10]:
datos=spark.read.csv("rl/cruise_ship_info.csv",header=True,inferSchema=True)

In [11]:
datos.printSchema(),datos.first()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



(None,
 Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55))

In [24]:
datos.corr("Age","crew")

-0.5306565039638852

In [25]:
datos.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [26]:
from pyspark.ml.feature import VectorAssembler

In [27]:
datos.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [29]:
constructor=VectorAssembler(inputCols=['Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density'],outputCol="features")

In [30]:
datosConVector=constructor.transform(datos)

In [31]:
datosConVector.show(1)

+---------+-----------+---+------------------+----------+------+------+-----------------+----+--------------------+
|Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|            features|
+---------+-----------+---+------------------+----------+------+------+-----------------+----+--------------------+
|  Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|[30.2769999999999...|
+---------+-----------+---+------------------+----------+------+------+-----------------+----+--------------------+
only showing top 1 row



In [36]:
datosFinales=datosConVector.select("features","crew")
datosFinales.show(truncate=False)

+------------------------------------------+----+
|features                                  |crew|
+------------------------------------------+----+
|[30.276999999999997,6.94,5.94,3.55,42.64] |3.55|
|[30.276999999999997,6.94,5.94,3.55,42.64] |3.55|
|[47.262,14.86,7.22,7.43,31.8]             |6.7 |
|[110.0,29.74,9.53,14.88,36.99]            |19.1|
|[101.353,26.42,8.92,13.21,38.36]          |10.0|
|[70.367,20.52,8.55,10.2,34.29]            |9.2 |
|[70.367,20.52,8.55,10.2,34.29]            |9.2 |
|[70.367,20.56,8.55,10.22,34.23]           |9.2 |
|[70.367,20.52,8.55,10.2,34.29]            |9.2 |
|[110.23899999999999,37.0,9.51,14.87,29.79]|11.5|
|[110.0,29.74,9.51,14.87,36.99]            |11.6|
|[46.052,14.52,7.27,7.26,31.72]            |6.6 |
|[70.367,20.52,8.55,10.2,34.29]            |9.2 |
|[70.367,20.52,8.55,10.2,34.29]            |9.2 |
|[86.0,21.24,9.63,10.62,40.49]             |9.3 |
|[110.0,29.74,9.51,14.87,36.99]            |11.6|
|[88.5,21.24,9.63,10.62,41.67]             |10.3|


In [37]:
datosEntrena,datosEvalua=datosFinales.randomSplit([0.8,0.2])

In [39]:
datosEvalua.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               34|
|   mean| 8.92764705882353|
| stddev|4.351869022945184|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [40]:
from pyspark.ml.regression import LinearRegression

In [41]:
lr=LinearRegression(labelCol="crew")

In [42]:
modelo=lr.fit(datosEntrena)

In [43]:
modelo.coefficients,modelo.intercept

(DenseVector([0.0281, -0.1747, 0.3472, 0.7634, -0.0038]), -0.41898845619385056)

In [44]:
modelo.summary.r2

0.9482198533810616

In [48]:
modelo.summary.meanAbsoluteError

0.5249236709198206

In [49]:
evalua=modelo.evaluate(datosEvalua)

In [50]:
evalua.r2

0.8663189973469438

In [51]:
evalua.predictions.show()

+--------------------+----+------------------+
|            features|crew|        prediction|
+--------------------+----+------------------+
|[3.341,0.66,2.79,...|0.59|0.5881973719656015|
|[10.0,2.08,4.4,1....| 1.6|1.6379959244364335|
|[25.0,7.76,6.22,3...|3.85|3.9124515887866282|
|[28.0,11.5,6.74,4...| 3.8| 3.660805976817465|
|[30.2769999999999...|3.73| 3.748547521902374|
|[45.0,11.78,7.54,...| 5.2| 5.307582511658734|
|[46.052,14.52,7.2...| 6.6| 6.285667396422517|
|[47.262,14.86,7.2...| 6.7| 6.372423649609528|
|[48.563,20.2,6.92...|6.71|  5.83670467075721|
|[50.76,17.48,7.54...|6.14| 7.134765719330082|
|[53.049,13.44,7.2...| 6.0| 6.257902121032514|
|[53.872,14.94,7.9...|6.12| 6.822555843675905|
|[63.0,14.4,7.77,7...|5.31|  6.86539342439462|
|[70.367,20.52,8.5...| 9.2| 8.600436214079572|
|[70.367,20.56,8.5...| 9.2| 8.608944887636024|
|[74.137,19.5,9.16...| 7.6| 8.738710494100225|
|[75.166,19.28,8.2...|7.66| 8.412926597801356|
|[76.8,19.6,8.79,9...|12.0|  8.60221524027189|
|[81.0,21.44,

In [52]:
evalua.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|0.001802628034398...|
|-0.03799592443643...|
|-0.06245158878662...|
| 0.13919402318253482|
|-0.01854752190237...|
|-0.10758251165873389|
|  0.3143326035774825|
| 0.32757635039047184|
|  0.8732953292427901|
| -0.9947657193300827|
| -0.2579021210325143|
| -0.7025558436759045|
| -1.5553934243946204|
|  0.5995637859204273|
|  0.5910551123639749|
| -1.1387104941002253|
| -0.7529265978013555|
|  3.3977847597281094|
|  0.6483843722728366|
| -0.6498732031190322|
+--------------------+
only showing top 20 rows



In [53]:
evalua.meanAbsoluteError,evalua.meanSquaredError,evalua.rootMeanSquaredError

(0.9293568721507447, 2.4572896372359576, 1.5675744439215502)

In [54]:
modeloDefinitivo=lr.fit(datosFinales)

In [55]:
modeloDefinitivo.summary.r2

0.9240699336931554

In [56]:
modeloDefinitivo.summary.predictions.show()

+--------------------+----+------------------+
|            features|crew|        prediction|
+--------------------+----+------------------+
|[30.2769999999999...|3.55| 3.800156189486712|
|[30.2769999999999...|3.55| 3.800156189486712|
|[47.262,14.86,7.2...| 6.7|6.4615020145191835|
|[110.0,29.74,9.53...|19.1|12.084253887450016|
|[101.353,26.42,8....|10.0|10.870892185173009|
|[70.367,20.52,8.5...| 9.2| 8.721442135297618|
|[70.367,20.52,8.5...| 9.2| 8.721442135297618|
|[70.367,20.56,8.5...| 9.2|  8.73138691540803|
|[70.367,20.52,8.5...| 9.2| 8.721442135297618|
|[110.238999999999...|11.5|11.005113289151343|
|[110.0,29.74,9.51...|11.6|12.068300134395411|
|[46.052,14.52,7.2...| 6.6| 6.377642571408785|
|[70.367,20.52,8.5...| 9.2| 8.721442135297618|
|[70.367,20.52,8.5...| 9.2| 8.721442135297618|
|[86.0,21.24,9.63,...| 9.3| 9.628906348405152|
|[110.0,29.74,9.51...|11.6|12.068300134395411|
|[88.5,21.24,9.63,...|10.3| 9.668713027551972|
|[70.367,20.52,8.5...| 9.2| 8.721442135297618|
|[88.5,21.24,

In [57]:
from pyspark.ml.feature import StringIndexer

In [60]:
datos.groupBy("Cruise_line").count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [61]:
indexer=StringIndexer(inputCol="Cruise_line",outputCol="indice")

In [68]:
model=indexer.fit(datos)#genera el indice
datosConIndice=model.transform(datos)#aplica el índice
datosConIndice.select("Cruise_line","indice").distinct().show()

+-----------------+------+
|      Cruise_line|indice|
+-----------------+------+
|            Costa|   5.0|
|        Norwegian|   4.0|
|              MSC|   7.0|
|             Star|   8.0|
|           Orient|  19.0|
|           Disney|  17.0|
|Regent_Seven_Seas|  10.0|
|  Royal_Caribbean|   0.0|
|              P&O|   9.0|
|         Windstar|  14.0|
|         Princess|   2.0|
|         Seabourn|  13.0|
|          Azamara|  16.0|
| Holland_American|   3.0|
|          Oceania|  12.0|
|        Celebrity|   6.0|
|          Crystal|  18.0|
|        Silversea|  11.0|
|           Cunard|  15.0|
|         Carnival|   1.0|
+-----------------+------+



In [69]:
datosConIndice.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|indice|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|  16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|  16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|   1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|   1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|   1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|   1.0|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.5

In [71]:
constructor2=VectorAssembler(inputCols=['Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'indice'],outputCol="features")

In [75]:
datosConVector2=constructor2.transform(datosConIndice)
datosFinales2=datosConVector2.select("features","crew")
datosFinales2.show(truncate=False)



+----------------------------------------------+----+
|features                                      |crew|
+----------------------------------------------+----+
|[30.276999999999997,6.94,5.94,3.55,42.64,16.0]|3.55|
|[30.276999999999997,6.94,5.94,3.55,42.64,16.0]|3.55|
|[47.262,14.86,7.22,7.43,31.8,1.0]             |6.7 |
|[110.0,29.74,9.53,14.88,36.99,1.0]            |19.1|
|[101.353,26.42,8.92,13.21,38.36,1.0]          |10.0|
|[70.367,20.52,8.55,10.2,34.29,1.0]            |9.2 |
|[70.367,20.52,8.55,10.2,34.29,1.0]            |9.2 |
|[70.367,20.56,8.55,10.22,34.23,1.0]           |9.2 |
|[70.367,20.52,8.55,10.2,34.29,1.0]            |9.2 |
|[110.23899999999999,37.0,9.51,14.87,29.79,1.0]|11.5|
|[110.0,29.74,9.51,14.87,36.99,1.0]            |11.6|
|[46.052,14.52,7.27,7.26,31.72,1.0]            |6.6 |
|[70.367,20.52,8.55,10.2,34.29,1.0]            |9.2 |
|[70.367,20.52,8.55,10.2,34.29,1.0]            |9.2 |
|[86.0,21.24,9.63,10.62,40.49,1.0]             |9.3 |
|[110.0,29.74,9.51,14.87,36.

In [72]:
datosConIndice.corr("indice","crew")

-0.48332562728617057